In [59]:
import fitz
import re
import requests
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter

PDF_URL = "https://www2.aneel.gov.br/cedoc/atren20211000.pdf"
LOCAL_PDF_PATH = r"../data/atren20211000.pdf"

# Document metadata
DOC_INFO_DEFAULTS = {
    "source_document_url": PDF_URL,
    "source_document_name": LOCAL_PDF_PATH,
    "document_type": "Resolução Normativa",
    "issuer": "ANEEL"
}

# Optimized patterns based on actual PDF structure
PATTERNS = {
    "titulo": re.compile(r"^\s*T[ÍI]TULO\s+([IVXLCDM]+)\s*$", re.IGNORECASE),
    "capitulo": re.compile(r"^\s*CAP[ÍI]TULO\s+([IVXLCDM]+)\s*$", re.IGNORECASE),
    "secao": re.compile(r"^\s*(?:SUB)?[Ss]e[çc][ãa]o\s+([IVXLCDM]+)\s*$", re.IGNORECASE),
    "artigo_start": re.compile(r"^\s*Art\.\s*(\d+(?:[A-Za-z])?º?)\s+(.*)", re.IGNORECASE),
    "paragrafo_start": re.compile(r"^\s*§\s*(\d+º?|único)\s+(.+)", re.IGNORECASE),
    "inciso_start": re.compile(r"^\s*([IVXLCDM]+(?:-[A-Z])?)\s*-\s*(.+)"),
    "alinea_start": re.compile(r"^\s*([a-z])\)\s*(.+)")
}

def clean_text_line(text: str) -> str:
    """Clean text by replacing special characters and normalizing whitespace."""
    replacements = {
        '\xa0': ' ',  # Non-breaking space
        '\xad': '',   # Soft hyphen
        '\u2013': '-', # En dash
        '\u2014': '-'  # Em dash
    }
    for k, v in replacements.items():
        text = text.replace(k, v)
    return text.strip()

def download_pdf_if_not_exists(url: str, local_path: str) -> bool:
    """Download PDF if it doesn't exist locally."""
    if os.path.exists(local_path):
        print(f"Arquivo {local_path} já existe. Usando arquivo local.")
        return True
    
    print(f"Baixando PDF de {url} para {local_path}...")
    try:
        response = requests.get(url, stream=True, timeout=30)
        response.raise_for_status()
        with open(local_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"PDF baixado com sucesso: {local_path}")
        return True
    except requests.RequestException as e:
        print(f"Erro ao baixar o PDF: {e}")
        return False

def build_full_hierarchical_path(metadata_dict: dict) -> str:
    """Build complete hierarchical path from metadata dictionary."""
    components = [
        metadata_dict.get("titulo_text"),
        metadata_dict.get("capitulo_text"),
        metadata_dict.get("secao_text"),
        metadata_dict.get("artigo_number"),
    ]
    return " > ".join(filter(None, components))

def parse_aneel_pdf(
        pdf_path: str,
        max_chunk_size: int = 1500,
        chunk_overlap: int = 200
) -> list[dict]:
    """
    Comprehensive parsing that captures ALL content and properly tracks hierarchy.
    """
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"Arquivo PDF não encontrado: {pdf_path}")
    
    all_chunks = []
    current_hierarchy = {
        "titulo_text": None,
        "capitulo_text": None,
        "secao_text": None,
        "artigo_number": None
    }
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", ". ", "; ", " ", ""],
        strip_whitespace=True
    )

    try:
        doc = fitz.open(pdf_path)
        print(f"Processando PDF: {pdf_path} com {len(doc)} páginas.")

        # First pass: track hierarchy as we go through the document
        full_text_with_hierarchy = []
        
        for page_num, page in enumerate(doc, 1):
            page_text = page.get_text()
            if not page_text.strip():
                continue
                
            lines = page_text.split('\n')
            for line in lines:
                line_clean = clean_text_line(line).strip()
                if not line_clean:
                    continue
                
                # Check for hierarchy updates
                titulo_match = PATTERNS["titulo"].match(line_clean)
                capitulo_match = PATTERNS["capitulo"].match(line_clean)
                secao_match = PATTERNS["secao"].match(line_clean)
                artigo_match = PATTERNS["artigo_start"].match(line_clean)
                
                if titulo_match:
                    current_hierarchy.update({
                        "titulo_text": f"TÍTULO {titulo_match.group(1)}",
                        "capitulo_text": None,
                        "secao_text": None,
                        "artigo_number": None
                    })
                elif capitulo_match:
                    current_hierarchy.update({
                        "capitulo_text": f"CAPÍTULO {capitulo_match.group(1)}",
                        "secao_text": None,
                        "artigo_number": None
                    })
                elif secao_match:
                    current_hierarchy.update({
                        "secao_text": f"Seção {secao_match.group(1)}",
                        "artigo_number": None
                    })
                elif artigo_match:
                    current_hierarchy["artigo_number"] = f"Art. {artigo_match.group(1)}"
                
                # Store text with current hierarchy context
                full_text_with_hierarchy.append({
                    "text": line_clean,
                    "page": page_num,
                    "hierarchy": current_hierarchy.copy()  # Important: copy the dict
                })
        
        # Second pass: create chunks with proper hierarchy metadata
        all_text = "\n".join([item["text"] for item in full_text_with_hierarchy])
        text_chunks = text_splitter.split_text(all_text)
        
        for idx, chunk in enumerate(text_chunks):
            # Find the most relevant hierarchy for this chunk
            chunk_metadata = DOC_INFO_DEFAULTS.copy()
            
            # Look for hierarchy elements in the chunk text
            best_hierarchy = {"titulo_text": None, "capitulo_text": None, "secao_text": None, "artigo_number": None}
            
            # Search for hierarchy markers in the original text with hierarchy
            for item in full_text_with_hierarchy:
                if item["text"] in chunk:
                    # Update with the most specific hierarchy found
                    if item["hierarchy"]["artigo_number"]:
                        best_hierarchy = item["hierarchy"].copy()
                        break
                    elif item["hierarchy"]["secao_text"] and not best_hierarchy["secao_text"]:
                        best_hierarchy = item["hierarchy"].copy()
                    elif item["hierarchy"]["capitulo_text"] and not best_hierarchy["capitulo_text"]:
                        best_hierarchy = item["hierarchy"].copy()
                    elif item["hierarchy"]["titulo_text"] and not best_hierarchy["titulo_text"]:
                        best_hierarchy = item["hierarchy"].copy()
            
            # Update metadata with hierarchy
            chunk_metadata.update(best_hierarchy)
            
            chunk_metadata.update({
                "chunk_index": idx,
                "total_chunks": len(text_chunks),
                "full_hierarchical_path": build_full_hierarchical_path(chunk_metadata)
            })

            all_chunks.append({
                "page_content": chunk,
                "metadata": chunk_metadata
            })
            
    except Exception as e:
        print(f"Erro ao processar o PDF: {e}")
        raise
    finally:
        if 'doc' in locals():
            doc.close()
            
    print(f"Total de chunks processados: {len(all_chunks)}")
    return all_chunks

# Test the function
if download_pdf_if_not_exists(PDF_URL, LOCAL_PDF_PATH):
    print("\n=== Testing Comprehensive Parsing ===")
    chunks = parse_aneel_pdf(LOCAL_PDF_PATH)
    print(f"Total chunks extracted: {len(chunks)}")

Arquivo ../data/atren20211000.pdf já existe. Usando arquivo local.

=== Testing Comprehensive Parsing ===
Processando PDF: ../data/atren20211000.pdf com 314 páginas.
Total de chunks processados: 549
Total chunks extracted: 549


In [ ]:
import streamlit as st
import os
import chromadb

from text_processor_2 import parse_aneel_pdf, download_pdf_if_not_exists, PDF_URL, LOCAL_PDF_PATH
from vector_db import initialize_vector_db, query_vector_db, COLLECTION_NAME
from chatbot_logic import generate_response_with_gemini

# --- Configuração ---
CHROMA_PERSIST_DIR = r"./chroma_db_data"
DB_READY_FLAG = "db_initialized.flag"  # Arquivo de flag para verificar se o banco de dados foi inicializado

# --- Função auxiliar para Checar/Construir o banco de dados ---
def ensure_db_is_ready():
    """
    Verifica se o banco de dados vetorial está pronto. Se não estiver, inicializa-o.
    """
    if not os.path.exists(DB_READY_FLAG):
        st.info("Base de dados vetorial não encontrada. Inicializando...")
        st.info("Baixando e processando o PDF da Resolução Normativa 1000 da ANEEL...")
        with st.spinner("Isso pode levar alguns minutos... ⏳"):
            # Download PDF if needed
            if download_pdf_if_not_exists(PDF_URL, LOCAL_PDF_PATH):
                # Parse PDF and extract chunks with hierarchy
                chunks_with_metadata = parse_aneel_pdf(LOCAL_PDF_PATH)
                
                # Extract just the text content for vector DB
                text_chunks = [chunk["page_content"] for chunk in chunks_with_metadata]
                
                # Extract metadata for vector DB
                metadatas = [chunk["metadata"] for chunk in chunks_with_metadata]
                
                # Initialize vector database with chunks and metadata
                initialize_vector_db_with_metadata(text_chunks, metadatas, persist_directory=CHROMA_PERSIST_DIR)
                
                # Create flag file
                with open(DB_READY_FLAG, 'w') as f:
                    f.write("Database initialized with PDF content")
            else:
                st.error("Erro ao baixar o PDF. Verifique sua conexão com a internet.")
                return
                
        st.success("Base de dados vetorial inicializada com sucesso! ✅")
    else:
        # Load existing collection
        try:
            client = chromadb.PersistentClient(path=CHROMA_PERSIST_DIR)
            collection = client.get_collection(name=COLLECTION_NAME)
            import vector_db
            vector_db.client = client  # Update global client
            vector_db.collection = collection  # Update global collection
            st.sidebar.success(f"Base de dados vetorial '{COLLECTION_NAME}' carregada com sucesso! ✅")
        except Exception as e:
            st.sidebar.error(f"Erro ao carregar a base de dados vetorial: {e}. Tentando recriar...")
            if os.path.exists(DB_READY_FLAG):
                os.remove(DB_READY_FLAG)  # Remove flag to force reinitialization
            ensure_db_is_ready()

def initialize_vector_db_with_metadata(documents: list[str], metadatas: list[dict], persist_directory: str = r"./chroma_db_data"):
    """
    Initialize vector database with documents and their metadata.
    """
    client = chromadb.PersistentClient(path=persist_directory)

    try:
        collection = client.get_collection(name=COLLECTION_NAME)
        print(f"Coleção '{COLLECTION_NAME}' já existe. Removendo para recriar...")
        client.delete_collection(name=COLLECTION_NAME)
    except:
        pass  # Collection doesn't exist, which is fine

    print(f"Criando nova coleção '{COLLECTION_NAME}'...")
    collection = client.create_collection(name=COLLECTION_NAME)

    # Create document IDs
    doc_ids = [f"doc_{i}" for i in range(len(documents))]
    
    # Add documents with metadata
    collection.add(
        documents=documents,
        ids=doc_ids,
        metadatas=metadatas
    )
    
    # Update global variables
    import vector_db
    vector_db.client = client
    vector_db.collection = collection
    
    print(f"Banco de dados vetorial inicializado com {len(documents)} documentos com metadados.")
    return collection

# --- Streamlit App ---
st.set_page_config(page_title="ANEEL Chatbot", page_icon=":robot_face:", layout="wide")
st.title("💬 Chatbot Inteligente de Leis da ANEEL (REN1000/2021)")
st.markdown("""
Bem-vindo(a)! Pergunte sobre a Resolução Normativa ANEEL nº 1000/2021.
Este chatbot utiliza a Google Generative AI para responder às suas perguntas com base nos textos da lei.
""")

# --- Sidebar para Chave de API do Gemini e Status do Banco de Dados ---
st.sidebar.header("Configurações")
api_key_input = st.sidebar.text_input(
    "Sua Chave de API Gemini (GOOGLE_API_KEY):",
    type="password",
    help="Obtenha sua chave em https://aistudio.google.com/app/apikey"
)

# Determina qual chave usar (entrada do usuário tem prioridade)
api_key = api_key_input or os.getenv("GOOGLE_API_KEY")

if api_key:
    # Configura a chave no ambiente se foi fornecida via input
    if api_key_input:
        os.environ["GOOGLE_API_KEY"] = api_key_input
    
    # Configura o cliente Gemini
    try:
        import google.generativeai as genai
        genai.configure(api_key=api_key)
        
        if api_key_input:
            st.sidebar.success("Chave de API configurada com sucesso! ✅")
        else:
            st.sidebar.info("Chave de API já configurada no ambiente. ✅")
    except Exception as e:
        st.sidebar.error(f"Erro ao configurar a chave de API: {e}")
else:
    st.sidebar.warning("Por favor, insira sua chave de API Gemini para continuar.")
    st.stop()

# Verifica se o banco de dados vetorial está pronto antes de permitir consultas
ensure_db_is_ready()

# Inicializa o histórico de mensagens
if "messages" not in st.session_state:
    st.session_state.messages = []

# Mostra o histórico de mensagens quando o app é recarregado
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Recebe a pergunta do usuário
if prompt := st.chat_input("Qual a sua pergunta sobre a REN 1000/2021 da ANEEL?"):
    # Adiciona a pergunta ao histórico
    st.session_state.messages.append({"role": "user", "content": prompt})
    
    # Exibe a pergunta na interface
    with st.chat_message("user"):
        st.markdown(prompt)
    
    # Mostra a resposta da IA em um container de mensagem
    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        message_placeholder.markdown("**Pensando...** 🧠")

        # 1. Consulta o banco de dados vetorial
        retrieved_chunks = query_vector_db(prompt, n_results=3)
        
        if not retrieved_chunks:
            full_response = "Desculpe, não consegui encontrar informações relevantes nos documentos consultados para responder à sua pergunta."
        else:
            # 2. Gera a resposta usando o modelo Gemini
            full_response = generate_response_with_gemini(prompt, retrieved_chunks)

        # Atualiza a mensagem com a resposta final
        message_placeholder.markdown(full_response)
        
        # Show sources with hierarchical information
        with st.expander("Ver fontes e contexto"):
            # Get the last query results with metadata
            if hasattr(st.session_state, 'last_query_results'):
                results = st.session_state.last_query_results
                for i, (doc, metadata) in enumerate(zip(results.get('documents', [[]])[0], results.get('metadatas', [[]])[0])):
                    st.caption(f"**Fonte {i+1}:**")
                    if metadata.get('full_hierarchical_path'):
                        st.caption(f"📍 **Localização:** {metadata['full_hierarchical_path']}")
                    st.caption(f"📄 **Conteúdo:** {doc[:200]}...")
                    st.divider()
            else:
                for i, doc in enumerate(retrieved_chunks):
                    st.caption(f"**Fonte {i+1}:** {doc[:200]}...")
    
    # Adiciona a resposta ao histórico
    st.session_state.messages.append({"role": "assistant", "content": full_response})

In [18]:
# data_preprocess.py
from bs4 import BeautifulSoup
import re
import requests

ANEEL_RES_PATH = r"../data/ren20211000.html"
ANEEL_RES_PATH_2 = r"../data/ren20211000_2.html"
def fetch_html_content(url: str) -> str | None:
    """
    Faz uma requisição HTTP para obter o conteúdo HTML de uma URL.
    :param url: URL do qual o conteúdo HTML será obtido.
    :return: Conteúdo HTML como string.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Verifica se a requisição foi bem-sucedida
        
        # Get encoding from response headers or detect it
        if response.encoding is None or response.encoding == 'ISO-8859-1':
            # Try to detect encoding from content
            response.encoding = response.apparent_encoding
        
        # If still no proper encoding, default to utf-8
        if response.encoding is None:
            response.encoding = 'utf-8'
            
        return response.text
    except requests.RequestException as e:
        print(f"Erro ao obter conteúdo HTML de {url}: {e}")
        return None

def clean_html(html_path: str) -> str:
    """
    Lê um arquivo HTML, remove elementos com estilo `text-decoration: line-through` e extrai o texto limpo.
    : param html_path: Caminho para o arquivo HTML a ser processado.
    : return: Texto limpo extraído do HTML.
    """
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'lxml')
    
    for strike in soup.find_all(style=re.compile('line-through')):
        strike.decompose()
    
    # Extract clean text
    return soup.get_text(separator='\n', strip=True)

if __name__ == "__main__":
    cleaned_text = clean_html(ANEEL_RES_PATH)
    print("\n--- Cleaned Text (first 500 characters) ---")
    print(cleaned_text[:500])

    with open(r"../data/cleaned_text.txt", "w", encoding='utf-8') as output_file:
        output_file.write(cleaned_text)
    print("\nCleaned text saved to '../data/cleaned_text.txt'.")    


--- Cleaned Text (first 500 characters) ---
AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL
RESOLUÇÃO NORMATIVA ANEEL Nº 1.000, DE 7 DE DEZEMBRO DE 2021(*)
Estabelece as Regras de Prestação do Serviço Público de Distribuição de Energia Elétrica; revoga as Resoluções Normativas ANEEL nº
414
, de 9 de setembro de 2010; nº
470
, de 13 de dezembro de 2011; nº
901
, de 8 de dezembro de 2020 e dá outras providências.
Decisão Judicial
Despacho 2.006/2024: Decisão Judicial - suspensos os efeitos referentes ao prazo de 60 (sessenta) ciclos estabeleci

Cleaned text saved to '../data/cleaned_text.txt'.


In [2]:
# text_processor.py

from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_text(text: str, chunk_size: int=1000, chunk_overlap: int=200) -> list[str]:
    """
    Divide o texto em pedaços menores com base no tamanho e na sobreposição especificados.
    : param text: String de texto a ser dividido.
    : param chunk_size: Tamanho máximo de cada pedaço de texto.
    : param chunk_overlap: Número de caracteres que se sobrepõem entre pedaços consecutivos.
    : return: Lista de pedaços de texto divididos.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        add_start_index=True
    )
    documents = text_splitter.create_documents([text])
    return [doc.page_content for doc in documents]

if __name__ == "__main__":
    with open(r"../data/cleaned_text.txt", "r", encoding='utf-8') as file:
        cleaned_text = file.read()

    chunks = split_text(cleaned_text, chunk_size=1000, chunk_overlap=200)
    print("\n--- Text Chunks (first 3 chunks) ---")
    for i, chunk in enumerate(chunks[:3]):
        print(f"Chunk {i+1}:\n{chunk}\n")


--- Text Chunks (first 3 chunks) ---
Chunk 1:
AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL
RESOLUÇÃO NORMATIVA ANEEL Nº 1.000, DE 7 DE DEZEMBRO DE 2021(*)
Estabelece as Regras de Prestação do Serviço Público de Distribuição de Energia Elétrica; revoga as Resoluções Normativas ANEEL nº
414
, de 9 de setembro de 2010; nº
470
, de 13 de dezembro de 2011; nº
901
, de 8 de dezembro de 2020 e dá outras providências.
Decisão Judicial
Despacho 2.006/2024: Decisão Judicial - suspensos os efeitos referentes ao prazo de 60 (sessenta) ciclos estabelecidos no inciso II do art. 323
.
Voto
Texto Compilado
O DIRETOR-GERAL DA AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL, no uso de suas atribuições regimentais, de acordo com a deliberação da Diretoria, tendo em vista o disposto na Lei nº 9.427, de 26 de dezembro de 1996, no Decreto nº 2.335, de 6 de outubro de 1997 e o que consta do Processo nº 48500.005218/2020-06, resolve:
TÍTULO I
PARTE GERAL
CAPÍTULO I
DAS DISPOSIÇÕES GERAIS
Seção I
Do Objeto e Âmb

In [3]:
# vector_db.py

import chromadb
from chromadb.utils import embedding_functions

client  = None
collection = None
COLLECTION_NAME = "aneel_collection"

def initialize_vector_db(documents: list[str], persist_directory: str=r"../chroma_db_data"):
    """
    Inicializa o banco de dados vetorial ChromaDB com os documentos fornecidos.
    O embedding de documentos é feito usando o modelo 'all-MiniLM-L6-v2'.
    : param documents: Lista de documentos a serem armazenados no banco de dados.
    : param persist_directory: Diretório onde o banco de dados será persistido.
    : return: Instância do cliente ChromaDB e coleção criada.
    """
    global client, collection
    
    client = chromadb.PersistentClient(path=persist_directory)

    try:
        collection = client.get_collection(name=COLLECTION_NAME)
        print(f"Coleção '{COLLECTION_NAME}' carregada com sucesso.")
    except:
        print(f"Coleção '{COLLECTION_NAME}' não encontrada. Criando nova coleção.")
        collection = client.create_collection(name=COLLECTION_NAME)
        print(f"Coleção '{COLLECTION_NAME}' criada com sucesso.")

        doc_ids = [f"doc_{i}" for i in range(len(documents))]
        collection.add(
            documents=documents,
            ids=doc_ids,
            metadatas=[{"source": f"doc_{i}"} for i in range(len(documents))],
        )
        print(f"Banco de dados vetorial inicializado com {len(documents)} documentos.")
    
    return collection

def query_vector_db(query_text: str, n_results: int=5) -> list[str]:
    """
    Consulta o banco de dados vetorial ChromaDB com uma string de consulta.
    : param query_text: Texto da consulta para buscar documentos relevantes.
    : param n_results: Número de resultados a serem retornados.
    : return: Lista de IDs dos documentos mais relevantes encontrados.
    """
    global collection
    if not collection:
        print("Erro: Coleção não inicializada. Por favor, chame initialize_vector_db primeiro.")
        try:
            global client
            client = chromadb.PersistentClient(path=r"../chroma_db_data")
            collection = client.get_collection(name=COLLECTION_NAME)
            print(f"Coleção '{COLLECTION_NAME}' carregada com sucesso.")
        except Exception as e:
            print(f"Erro ao carregar a coleção: {e}")
            return []
    
    if not query_text:
        return []
    
    results = collection.query(
        query_texts=[query_text],
        n_results=n_results
    )
    return results['documents'][0]  if results and results['documents'] else []

if __name__ == "__main__":
    with open(r"../data/cleaned_text.txt", "r", encoding='utf-8') as file:
        cleaned_text = file.read()
    
    chunks = split_text(cleaned_text)
    initialize_vector_db(chunks)
    print("\n--- Vector Database Initialized ---")

    if collection:
        query = "Quais são as principais mudanças na regulamentação de energia?"
        print(f"\nQuery: {query}")
        retrieved_docs = query_vector_db(query, n_results=2)
        if retrieved_docs:
            print("\nRetrieved Documents:")
            for doc in retrieved_docs:
                print(f"- {doc[:300]}...")
        else:
            print("Nenhum documento encontrado para a consulta.")
    else:
        print("Falha ao inicializar a coleção. Verifique os logs para mais detalhes.")

Coleção 'aneel_collection' carregada com sucesso.

--- Vector Database Initialized ---

Query: Quais são as principais mudanças na regulamentação de energia?

Retrieved Documents:
- Seção III
Do Contrato de Compra de Energia Regulada – CCER
Art. 162. O CCER deve conter as cláusulas gerais do art. 145 e, caso aplicáveis, as do art. 132, além de outras consideradas essenciais, observando as demais disposições deste Capítulo.
Art. 163. O montante de energia elétrica contratado por...
- Art. 29. O consumidor e demais usuários devem observar em suas instalações as normas e padrões da distribuidora, as normas da Associação Brasileira de Normas Técnicas - ABNT e as normas dos órgãos oficiais competentes, naquilo que for aplicável e não contrariar à regulação da ANEEL.
Art. 30. O consu...


General text: 500-1000 characters

Technical docs: 300-800 characters

Code: 200-500 characters (split at logical boundaries)

In [4]:
retrieved_docs = query_vector_db("Qual é o periodo de testes para unidade consumidara?", n_results=2)
if retrieved_docs:
    print("\nRetrieved Documents:")
    for doc in retrieved_docs:
        print(f"- {doc}...")


Retrieved Documents:
- Seção XI
Do Período de Testes e Ajustes
Art. 311. A distribuidora deve aplicar o período de testes para unidade consumidora para permitir a adequação da demanda contratada de consumo e a escolha da modalidade tarifária, nas seguintes situações: (
Redação dada pela REN ANEEL 1.059, de 07.02.2023
)
I - início do fornecimento de energia elétrica;
II - mudança para faturamento aplicável à unidade consumidora do grupo
A, cuja opção anterior tenha sido por faturamento do grupo B;
III - enquadramento na modalidade tarifária horária azul; e
IV - acréscimo de demanda, quando maior que 5% da contratada.
Parágrafo único. Quando do enquadramento na modalidade tarifária horária azul, o período de testes abrangerá exclusivamente o montante contratado para o posto tarifário ponta.
Art. 312. O período de testes deve ter duração de 3 ciclos consecutivos e completos de faturamento.
Parágrafo único.
A distribuidora pode prorrogar o período de testes, mediante solicitação fundament

In [ ]:
# chatbot_logic.py
import google.generativeai as genai
import os

from dotenv import load_dotenv
load_dotenv()

try:
    if not os.getenv("GOOGLE_API_KEY"):
        raise ValueError("A variável de ambiente GOOGLE_API_KEY não está definida.")
    genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
except ValueError as e:
    print(f"Erro de configuração: {e}. Garanta que a variável de ambiente GOOGLE_API_KEY esteja definida.")

def generate_response_with_gemini(query: str, context_chuncks: list[str]) -> str:
    """
    Gera uma resposta usando o modelo Gemini da Google Generative AI, incorporando o contexto dos documentos recuperados.
    : param query: Texto da consulta do usuário.
    : param context_chuncks: Lista de pedaços de texto que fornecem contexto adicional para a resposta.
    : return: Resposta gerada pelo modelo Gemini.
    """
    if not os.getenv("GOOGLE_API_KEY"):
        raise ValueError("A variável de ambiente GOOGLE_API_KEY não está definida.")
    
    # Prepara o contexto para a consulta
    context_str = "\n\n---\n\n".join(context_chuncks)
    prompt = f"""
    Você é um assistente de IA especializado em leis e regulamentos brasileiros da ANEEL.
    Sua tarefa é responder à pergunta do usuário com base estritamente nos trechos de lei fornecidos abaixo.
    Não utilize conhecimento externo. Se a resposta não puder ser encontrada nos trechos fornecidos,
    declare explicitamente que a informação não está disponível nos documentos consultados.
    Seja conciso e direto ao ponto. Responda em português brasileiro.
    
    **Contexto (Trechos da Lei):**
    {context_str}
    
    **Pergunta do Usuário:**
    {query}

**Sua Resposta:**
"""
    try:
        model = genai.GenerativeModel(model_name="gemini-1.5-flash-8b")
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Erro ao gerar resposta com Gemini: {e}")
        return "Desculpe, ocorreu um erro ao processar sua consulta. Tente novamente mais tarde."

test_user_query = "O que deve constar na segunda via de fatura?"
retrieved_docs = query_vector_db(test_user_query, n_results=2)
if retrieved_docs:
    print("\nRetrieved Documents for Test Query:")
    for doc in retrieved_docs:
        print(f"- {doc[:300]}...")

    response = generate_response_with_gemini(test_user_query, retrieved_docs)
    print("\n--- Response from Gemini ---")
    print(response)


Retrieved Documents for Test Query:
- § 3º A distribuidora deve disponibilizar a fatura de energia elétrica completa no espaço reservado de atendimento pela internet, independentemente da opção pelo resumo da fatura.
Art. 329. A distribuidora deve prestar ao consumidor e demais usuários esclarecimentos sobre os tributos, subvenções e in...
- VII - formalidades e exigências que sejam incompatíveis com a boa-fé, excessivamente onerosas ou cujo custo econômico ou social seja superior ao risco envolvido.
Parágrafo único. No caso de núcleo urbano informal consolidado, nos termos da Lei nº 13.465, de 11 de julho de 2017, a comprovação de poss...

--- Response from Gemini ---
Todas as informações da primeira via, com destaque à expressão "segunda via".

